In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.style as style
from datetime import timedelta
import datetime as dt
import time
import os
os.system("pip install fuzzywuzzy")


%matplotlib inline

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [7]:
# Import data
df = pd.read_excel('C:/Users/saika/Desktop/online_retail_II.xlsx')


In [9]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      417534 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 28.7+ MB


In [11]:
df.dropna(inplace = True)

In [12]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [14]:
df.describe()

,Quantity,Price,Customer ID
count,417534.000000,417534.000000,417534.000000
mean,12.758815,3.887547,15360.645478
std,101.220424,71.131797,1680.811316
min,-9360.000000,0.000000,12346.000000
25%,2.000000,1.250000,13983.000000
50%,4.000000,1.950000,15311.000000
75%,12.000000,3.750000,16799.000000
max,19152.000000,25111.090000,18287.000000


In [15]:
df.corr()

,Quantity,Price,Customer ID
Quantity,1.000000,-0.004112,-0.011577
Price,-0.004112,1.000000,-0.003266
Customer ID,-0.011577,-0.003266,1.000000


In [ ]:
# Data Mining To check frequently bought items : 


In [66]:

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

def create_basket(country_filter):
    basket = (df[df['Country'] == country_filter]
          .groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))
    return basket

In [70]:
country_filter = "Germany"
basket_uk = create_basket(country_filter)
basket = basket_uk.applymap(encode_units)
basket.drop('POSTAGE', inplace=True, axis=1)

In [71]:
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)


In [72]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(RED TOADSTOOL LED NIGHT LIGHT),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.120858,0.192982,0.052632,0.435484,2.256598,0.029308,1.429574
1,(ROUND SNACK BOXES SET OF4 WOODLAND ),(RED TOADSTOOL LED NIGHT LIGHT),0.192982,0.120858,0.052632,0.272727,2.256598,0.029308,1.208821
2,(RED TOADSTOOL LED NIGHT LIGHT),(WOODLAND CHARLOTTE BAG),0.120858,0.146199,0.054581,0.451613,3.089032,0.036912,1.556932
3,(WOODLAND CHARLOTTE BAG),(RED TOADSTOOL LED NIGHT LIGHT),0.146199,0.120858,0.054581,0.373333,3.089032,0.036912,1.402887
4,(ROUND SNACK BOXES SET OF4 WOODLAND ),(ROUND SNACK BOXES SET OF 4 FRUITS ),0.192982,0.126706,0.099415,0.515152,4.065734,0.074963,1.801170


    # Content based recommendation

In [91]:
df1 = df[df['Description'].notna()]
df1['Description']=df1['Description'].str.strip()

    # Convert to list
unique_brand = df1.Description.unique().tolist()

In [92]:
sorted(unique_brand)[:20]

['10 COLOUR SPACEBOY PEN',
 '11 PC CERAMIC TEA SET POLKADOT',
 '12 ASS ZINC CHRISTMAS DECORATIONS',
 '12 COLOURED PARTY BALLOONS',
 '12 DAISY PEGS IN WOOD BOX',
 '12 EGG HOUSE PAINTED WOOD',
 '12 IVORY ROSE PEG PLACE SETTINGS',
 '12 MESSAGE CARDS WITH ENVELOPES',
 '12 MINI TOADSTOOL PEGS',
 '12 PENCIL SMALL TUBE WOODLAND',
 '12 PENCILS SMALL TUBE POSY',
 '12 PENCILS SMALL TUBE RED RETROSPOT',
 '12 PENCILS SMALL TUBE RED SPOTTY',
 '12 PENCILS SMALL TUBE SKULL',
 '12 PENCILS TALL TUBE POSY',
 '12 PENCILS TALL TUBE RED RETROSPOT',
 '12 PENCILS TALL TUBE RED SPOTTY',
 '12 PENCILS TALL TUBE SKULLS',
 '12 PENCILS TALL TUBE WOODLAND',
 '12 PINK HEN+CHICKS IN BASKET']

In [93]:
from fuzzywuzzy import process, fuzz
process.extract('PEACE WOODEN BLOCK LETTERS', unique_brand, scorer=fuzz.token_sort_ratio)


[('PEACE WOODEN BLOCK LETTERS', 100),
 ('WELCOME  WOODEN BLOCK LETTERS', 89),
 ('NOEL WOODEN BLOCK LETTERS', 86),
 ('JOY WOODEN BLOCK LETTERS', 80),
 ('PEACE SMALL WOOD LETTERS', 72)]

In [94]:
#Create tuples of brand names, matched brand names, and the score
score_sort = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand,     scorer=fuzz.token_sort_ratio)]
#Create a dataframe from the tuples
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])
similarity_sort.head()

,brand_sort,match_sort,score_sort
0,15CM CHRISTMAS GLASS BALL 20 LIGHTS,15CM CHRISTMAS GLASS BALL 20 LIGHTS,100
1,15CM CHRISTMAS GLASS BALL 20 LIGHTS,CHRISTMAS LIGHTS 10 VINTAGE BAUBLES,66
2,15CM CHRISTMAS GLASS BALL 20 LIGHTS,CHRISTMAS LIGHTS 10 SANTAS,62
3,15CM CHRISTMAS GLASS BALL 20 LIGHTS,4 PINK FLOCK CHRISTMAS BALLS,60
4,15CM CHRISTMAS GLASS BALL 20 LIGHTS,CHRISTMAS LIGHTS 10 REINDEER,60


In [95]:
similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])
similarity_sort.head()

,brand_sort,match_sort,score_sort,sorted_brand_sort
0,15CM CHRISTMAS GLASS BALL 20 LIGHTS,15CM CHRISTMAS GLASS BALL 20 LIGHTS,100,15CM CHRISTMAS GLASS BALL 20 LIGHTS
1,15CM CHRISTMAS GLASS BALL 20 LIGHTS,CHRISTMAS LIGHTS 10 VINTAGE BAUBLES,66,15CM CHRISTMAS GLASS BALL 20 LIGHTS
2,15CM CHRISTMAS GLASS BALL 20 LIGHTS,CHRISTMAS LIGHTS 10 SANTAS,62,15CM CHRISTMAS GLASS BALL 20 LIGHTS
3,15CM CHRISTMAS GLASS BALL 20 LIGHTS,4 PINK FLOCK CHRISTMAS BALLS,60,15CM CHRISTMAS GLASS BALL 20 LIGHTS
4,15CM CHRISTMAS GLASS BALL 20 LIGHTS,CHRISTMAS LIGHTS 10 REINDEER,60,15CM CHRISTMAS GLASS BALL 20 LIGHTS


In [97]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['brand_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

In [98]:
high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)


,,match_sort
brand_sort,score_sort,
BLUE PATCH PURSE PINK HEART,100,PINK PATCH PURSE BLUE HEART
NUMBER TILE COTTAGE GARDEN 7,100,"NUMBER TILE COTTAGE GARDEN, 7"
RIBBON REEL MAKING SNOWMEN,100,"RIBBON REEL, MAKING SNOWMEN"
NUMBER TILE COTTAGE GARDEN 5,100,"NUMBER TILE COTTAGE GARDEN, 5"
NUMBER TILE COTTAGE GARDEN 4,100,"NUMBER TILE COTTAGE GARDEN, 4"
NUMBER TILE COTTAGE GARDEN 2,100,"NUMBER TILE COTTAGE GARDEN, 2"
ROSES REGENCY TEACUP AND SAUCER,100,TEACUP AND SAUCER REGENCY ROSES
NUMBER TILE COTTAGE GARDEN 1,100,"NUMBER TILE COTTAGE GARDEN, 1"
NUMBER TILE COTTAGE GARDEN 0,100,"NUMBER TILE COTTAGE GARDEN, 0"


In [103]:
df1[(df1['Description'] == 'RED RETROSPOT MUG') | (df1['Description'] == 'RED RETROSPOT PURSE')].sort_values(['Description'])


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
151095,503790,21844,RED RETROSPOT MUG,1,2010-04-07 12:45:00,2.95,13174.0,United Kingdom
379115,525928,21844,RED RETROSPOT MUG,6,2010-10-07 16:48:00,2.95,16072.0,United Kingdom
379858,525967,21844,RED RETROSPOT MUG,6,2010-10-08 10:00:00,2.95,14243.0,United Kingdom
380893,526096,21844,RED RETROSPOT MUG,6,2010-10-08 12:35:00,2.95,13615.0,United Kingdom
381391,526141,21844,RED RETROSPOT MUG,36,2010-10-08 14:44:00,2.55,13406.0,United Kingdom
382740,526207,21844,RED RETROSPOT MUG,2,2010-10-10 13:31:00,2.95,15547.0,United Kingdom
385479,526417,21844,RED RETROSPOT MUG,6,2010-10-11 14:32:00,2.95,15111.0,United Kingdom
389222,526853,21844,RED RETROSPOT MUG,12,2010-10-13 12:48:00,2.95,12540.0,Spain
392852,527130,21844,RED RETROSPOT MUG,12,2010-10-15 09:31:00,2.95,15296.0,United Kingdom
378732,525902,21844,RED RETROSPOT MUG,12,2010-10-07 15:44:00,2.95,16166.0,United Kingdom
